# Golden/Death Cross (MM50d-MM200d)

Se analiza el mercado desde la perspectiva del cruce de medias móviles de 50 y 200 sesiones (también llamado "Golden Cross", al alza, o bien "Death Cross" a la baja. Permite observar estado de situación y también últimas señales vigentes.

# INDICE DE FUNCIONES:
- analyzeGoldenDeath(activos, timeframe):
    Esta función analiza el estado de las medias móviles exponenciales de 50 y 200 ruedas para el listado de activos solicitado.
    Evalúa dos métricas: 
     - Distancia: La posición de la MM de 50 ruedas respecto de la MM de 200 ruedas (si está por encima se considera en
    posición "Alcista" o de "Golden Cross", y si está por debajo se considera posición "Bajista" o "Death Cross").
    - crossover: se analiza la posición de la "Distancia" (simil línea macd) respecto de una EMA de 30 sesiones de la Distancia (simil
    línea señal del macd). Si la distancia actual está por encima de ese promedio móvil, el crossover activado es de alineación "Alcista",
    sino, "Bajista".
- getGoldenDeathSignals(activos, timeframe):
    Esta función devuelve los cruces vigentes de las medias móviles exponenciales de 50 y 200 sesiones del listado de activos
    solicitado, así como el sentido del cruce, la fecha de la señal, el precio de la señal, el rendimiento acumulado a la fecha
    y el rendimiento acumulado, todo ordenado en forma descendente por cantidad de días transcurridos desde la señal.

In [1]:
# FUNCIONES NECESARIAS PARA OBTENER DATA FINANCIERA:

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

def getDataYf(ticker, tipo, interval, data_from = None, data_to = None, period = None):
    """
    Es una función para descargar market data de Yahoo Finance con la librería yfinance.
    
    ## Inputs:
        >ticker: el nombre del ticker.
        >tipo: si es "no end" no se indica hasta cuándo (data_to), se obtiene hasta el último día disponible. Si es "end" es
        necesario indicar hasta cuánto (data_to). En ambos casos hay que indicar desde qué fecha (data_from). Si es "period" 
        no se indica ni desde cuándo ni hasta cuándo, sólo el argumento "period" con la cantidad de tiempo a obtener.
        >now : si es True, no se indica hasta cuándo (data_to), se obtiene hasta el último día disponible. Si se indica False, es
        necesario indicar hasta cuánto (data_to).
        >interval: el timeframe (ej. 1mo, 1h, 1d, 1wk, etc)
        >data_from: data desde qué fecha.
        >data_to: data hasta qué fecha (no inclusive el día). Sólo es aplicable si now == True.
        >period : en caso de tipo = "period", se pasa este argumento que refiere a la cantidad de tiempo a obtener. Ej. 1y, 2y, 3y, etc.
        
    ## Outputs:
        >series OHLC ajustadas del ticker.
    """
    import yfinance as yf
    import pandas as pd
    
    if tipo == "no end":
        data = yf.download(ticker, start = data_from, interval = interval, progress = False, auto_adjust = True)
    elif tipo == "end":
        data = yf.download(ticker, start = data_from, end = data_to, interval = interval, progress = False, auto_adjust = True)
    elif tipo == "period":
        data = yf.download(ticker, interval = interval, period = period, progress = False, auto_adjust = True)
    return data



def getDataYfMulti(activos, tipo, interval, data_from = None, data_to = None, period = None, swap = True):
    """
    Función para hacer batch requests (varios tickers a la vez), que será la fx que más voy a utilizar para market data.
    
    ## Inputs:
        >tickers: es una lista con los tickers de los cuales se va a obtener market data.
        >tipo: si es "no end" no se indica hasta cuándo (data_to), se obtiene hasta el último día disponible. Si es "end" es
        necesario indicar hasta cuánto (data_to). En ambos casos hay que indicar desde qué fecha (data_from). Si es "period" 
        no se indica ni desde cuándo ni hasta cuándo, sólo el argumento "period" con la cantidad de tiempo a obtener.
        >now : si es True, no se indica hasta cuándo (data_to), se obtiene hasta el último día disponible. Si se indica False, es
        necesario indicar hasta cuánto (data_to).
        >interval: el timeframe (ej. 1mo, 1h, 1d, 1wk, etc)
        >data_from: data desde qué fecha.
        >data_to: data hasta qué fecha (no inclusive el día). Sólo es aplicable si now == True.
        >period : en caso de tipo = "period", se pasa este argumento que refiere a la cantidad de tiempo a obtener. Ej. 1y, 2y, 3y, etc.
        >swap : si es True, divide el df en tickers y cada uno tiene su OHLC. Si es False, tenemos cada columna OHLC y dentro todos los tickers.
        
    ## Outputs:
        >series OHLC ajustadas del ticker.
    """
    import yfinance as yf
    import pandas as pd
    
    lideres_arg = ["ALUA.BA", "BBAR.BA", "BMA.BA", "BYMA.BA", "CEPU.BA", "COME.BA", "CRES.BA", "CVH.BA", "EDN.BA", 
                   "GGAL.BA", "LOMA.BA", "MIRG.BA", "PAMP.BA", "SUPV.BA", "TECO2.BA", "TGNO4.BA", "TGSU2.BA", "TRAN.BA", 
                   "TXAR.BA", "VALO.BA", "YPFD.BA"]

    general_arg = ["AGRO.BA", "AUSO.BA", "BHIP.BA", "BOLT.BA", "BPAT.BA", "CADO.BA", "CAPX.BA", "CARC.BA", "CECO2.BA", 
                   "CELU.BA", "CGPA2.BA", "CTIO.BA", "DGCU2.BA", "FERR.BA", "FIPL.BA", "GAMI.BA", "GCDI.BA", "GCLA.BA", 
                   "HARG.BA", "HAVA.BA", "INVJ.BA", "IRSA.BA", "LEDE.BA", "LONG.BA", "METR.BA", "MOLA.BA", "MOLI.BA", 
                   "MORI.BA", "OEST.BA", "PATA.BA", "RICH.BA", "RIGO.BA", "SAMI.BA", "SEMI.BA"]

    cedears = ["AAL", "AAPL", "ABBV", "ABEV", "ABNB", "ABT", "ADBE", "ADGO", "ADI", "ADP", "AEM", "AIG", "AMAT", "AMD", 
               "AMGN", "AMZN", "AOCA", "ARCO", "ARKK", "ASR", "AUY", "AVGO", "AXP", "AZN", "BA", "BA.C", "BABA", "BB", 
               "BBD", "BBV", "BCS", "BHP", "BIDU", "BIIB", "BIOX", "BITF", "BK", "BMY", "BNG", "BP", "BRFS", "BRKB", "BSBR", 
               "C", "CAAP", "CAH", "CAR", "CAT", "CBRD", "CDE", "CL", "COIN", "COST", "CRM", "CS", "CSCO", "CVX", "CX", "DD", 
               "DE", "DESP", "DIA", "DISN", "DOCU", "DOW", "E", "EA", "EBAY", "EEM", "EFX", "ERIC", "ERJ", "ETSY", "EWZ", "F", 
               "FCX", "FDX", "FMX", "FSLR", "GE", "GFI", "GGB", "GILD", "GLOB", "GLW", "GM", "GOLD", "GOOGL", "GPRK", "GRMN", 
               "GS", "HAL", "HD", "HL", "HMC", "HMY", "HOG", "HON", "HPQ", "HSBC", "HSY", "HUT", "HWM", "IBM", "IFF", "INTC", 
               "ITUB", "IWM", "JD", "JMIA", "JNJ", "JPM", "KMB", "KO", "KOFM", "LLY", "LMT", "LRCX", "LVS", "LYG", "MA", "MCD", 
               "MDT", "MELI", "META", "MMM", "MO", "MOS", "MRK", "MSFT", "MSI", "MSTR", "MU", "NEM", "NFLX", "NGG", "NIO", "NKE", 
               "NOKA", "NTCO", "NTES", "NUE", "NVDA", "NVS", "ORAN", "ORCL", "OXY", "PAAS", "PAC", "PANW", "PBI", "PBR", "PCAR", 
               "PEP", "PFE", "PG", "PHG", "PKS", "PSX", "PYPL", "QCOM", "QQQ", "RBLX", "RIO", "RTX", "SAN", "SAP", "SATL", "SBUX", 
               "SCCO", "SE", "SHEL", "SHOP", "SI", "SID", "SLB", "SNAP", "SNOW", "SONY", "SPGI", "SPOT", "SPY", "SQ", "SYY", "T", 
               "TEFO", "TEN", "TGT", "TM", "TMO", "TRIP", "TRVV", "TSLA", "TSM", "TTE", "TV", "TWLO", "TXN", "TXR", "UAL", "UBER", 
               "UGP", "UL", "UNH", "UNP", "UPST", "USB", "V", "VALE", "VIST", "VIV", "VOD", "VZ", "WBA", "WFC", "WMT", "X", "XLE", 
               "XLF", "XOM", "XP", "YY", "ZM"]

    adrs = ["BBAR", "BMA", "CEPU", "CRESY", "EDN", "GGAL", "IRS", "LOMA", "PAM", "SUPV", "TEO", "TGS", "TS", "TX", "YPF"]

    sectors = ["XLC", "XLP", "XLY", "XLF", "XLV", "XLI", "XLRE", "XLU", "XBI", "XLB", "XLK", "XLE"]
    
    precarga = ["lideres", "general", "cedears", "adrs", "sectores"]
    precarga_dict = {"lideres" : lideres_arg, "general" : general_arg, "cedears" : cedears, "adrs" : adrs, "sectores" : sectors}
    
    if activos in precarga:
        activos = precarga_dict[activos]
    
    if tipo == "no end":
        data = yf.download(activos, start = data_from, interval = interval, progress = False, auto_adjust = True)
    elif tipo == "end":
        data = yf.download(activos, start = data_from, end = data_to, interval = interval, progress = False, auto_adjust = True)
    elif tipo == "period":
        data = yf.download(activos, interval = interval, period= period, progress = False, auto_adjust = True)
    
    if swap:
        #data = data.swaplevel(i = 0, j = 1, axis = 1)
        # Algoritmo para procesar el MultipleTicker download de yfinance
        dicto = {}
        low = data["Low"]
        high = data["High"]
        close = data["Close"]
        open = data["Open"]
        volume = data["Volume"]

        tickers = list(data["Close"].columns)

        for ticker in tickers:
            dicto[ticker] = {
                "Open" : open[ticker],
                "High" : high[ticker],
                "Low" : low[ticker],
                "Close" : close[ticker],
                "Volume" : volume[ticker]
            }

            dicto[ticker] = pd.DataFrame(dicto[ticker])
        return dicto
    return data

In [3]:
# FUNCIONALIDADES PRINCIPALES DEL MÓDULO:

def analyzeGoldenDeath(activos, timeframe):
    """
    Esta función analiza el estado de las medias móviles exponenciales de 50 y 200 ruedas para el listado de activos solicitado.
    Evalúa dos métricas: 
     - Distancia: La posición de la MM de 50 ruedas respecto de la MM de 200 ruedas (si está por encima se considera en
    posición "Alcista" o de "Golden Cross", y si está por debajo se considera posición "Bajista" o "Death Cross").
    - crossover: se analiza la posición de la "Distancia" (simil línea macd) respecto de una EMA de 30 sesiones de la Distancia (simil
    línea señal del macd). Si la distancia actual está por encima de ese promedio móvil, el crossover activado es de alineación "Alcista",
    sino, "Bajista".
                       
    # Inputs:
        - activos: es una lista de activos a analizar. Ej.: adrs = ["GGAL", "BBAR", "LOMA", "CEPU"]. También hay sets de activos preconfigurados en la función:
            - "lideres": si se desea cargar todo el panel lider de Argentina;
            - "general": si se desea cargar todo el panel general de Argentina;
            - "adrs": si se desea cargar todo el listado de ADRs argentinos (en usd);
            - "sectors": si se desea cargar el listado de sectores de la economía (ETFs representativos de USA);
            - "cedears": si se desea cargar todo el listado de certificados de depósito de activos del exterior que cotizan en Argentina (puede demorar bastante).
        - timeframe: es la duración de cada sesión/vela (compresión temporal). Puede ser "diario", "semanal" o "mensual".
        
    # Outputs:
        - tabla_final_activos_en_filas: es un DF resumen con los activos en las filas y las variables en las columnas.
        - tabla_final_activos_en_columnas: es un DF resumen con los activos en las columnas y las variables en las filas.
        - df_distancia: tabla con los valores numéricos de la métrica "Distancia" explicada antes;
        - df_crossover: tabla con los valores categóricos de la métrica "crossover" explicada antes;
        - estilado: tabla_final_activos_en_filas estilada con colores rojos y verdes según tendencia y crossover.
    """
    
    import pandas as pd
    import pandas_ta as ta
    import yfinance as yf
    
    if timeframe == "diario":
        interval = "1d"
        period = "3y"
    elif timeframe == "semanal":
        interval = "1wk"
        period = "10y"
    elif timeframe == "mensual":
        interval = "1mo"
        period = "15y"
    else:
        return "timeframe incorrecto, elegir 'diario', 'semanal' o 'mensual'"
    
    data = getDataYfMulti(activos, tipo = "period", interval = interval, period = period, swap = False)["Close"]
    data = pd.DataFrame(data)
    data.dropna(inplace = True)

    # Defino datos de estrategia
    MyStrategy = ta.Strategy(
    name = "goldenCross",
    ta = [
        {"kind" : "ema", "close" : "close", "length" : 50},
        {"kind" : "ema", "close" : "close", "length" : 200},
    ]
    )
    
    df_distancia = pd.DataFrame(index = data.index)  # Es el feature numerico (EMA50/EMA200 - 1) (Variación no porcentual)
    df_crossover = pd.DataFrame(index = data.index)  # Es el feature categorico de crossover de tend: si últimos 3 valores de df_distancia son crecientes, validamos tendencia creciente.            

    tendencia = pd.DataFrame()
    crossover = pd.DataFrame()

    #df_resumen = pd.DataFrame() # ==> Resumen de todos los activos listados en una sola tabla.
    dict_resumenPorActivo = {}  # ==> Resumen por activo (un diccionario de diccionarios).

    i = 0
    
    for activo in data.columns:
        try:
            datat = pd.DataFrame(data[activo])
            datat.rename(columns = {activo : "Close"}, inplace = True)   # Tengo que renombrar el ticker por "Close", sino no se calcula la MACD.
            datat.ta.strategy(MyStrategy)
            datat.dropna(inplace = True)
            datat["distancia"] = (datat["EMA_50"] / datat["EMA_200"]) - 1
            datat["MMdistancia"] = datat["distancia"].ewm(span = 30).mean()   # Similar a la línea señal del MACD, pero con 30 ruedas
            
            datat["diferencia"] = (datat["distancia"] / datat["MMdistancia"]) - 1
            
            datat["posicion"] = np.where(datat["diferencia"] > 0, 1, -1)
            datat["change"] = np.where(datat["posicion"] > datat["posicion"].shift(1), "Compra", 
                                            np.where(datat["posicion"] < datat["posicion"].shift(1), "Venta", ""))
            
            datat["crossover"] = np.where(datat["diferencia"] > 0, "Alcista", "Bajista")
            
            df_distancia[activo] = datat["distancia"] 
            df_crossover[activo] = datat["crossover"]
            df_distancia = df_distancia.dropna()
            df_crossover = df_crossover.dropna() 


            if df_distancia[activo][-1] >= 0:
                if df_crossover[activo][-1] == "Alcista":
                    dict_resumenPorActivo[activo] = {"tendencia" : "Alcista", "crossover" : "Alcista"}
                elif df_crossover[activo][-1] == "Bajista":
                    dict_resumenPorActivo[activo] = {"tendencia" : "Alcista", "crossover" : "Bajista"}
            
            elif df_distancia[activo][-1] < 0:
                if df_crossover[activo][-1] == "Bajista":
                    dict_resumenPorActivo[activo] = {"tendencia" : "Bajista", "crossover" : "Bajista"}
                elif df_crossover[activo][-1] == "Alcista":
                    dict_resumenPorActivo[activo] = {"tendencia" : "Bajista", "crossover" : "Alcista"}

            tabla_final_activos_en_columnas = pd.DataFrame(dict_resumenPorActivo)
            tabla_final_activos_en_filas = tabla_final_activos_en_columnas.transpose()
            i += 1
            print(f"[TF {timeframe}] Procesando {i} de {len(data.columns)} activos ...")

        except:
            pass
    
    tabla = tabla_final_activos_en_filas
    
    def set_styles(tabla):
        def highlight_tend(val, color_if_true, color_if_false):
            color = color_if_true if val == "Alcista" else color_if_false
            return f"background-color: {color}"

        styler = tabla.style.applymap(highlight_tend, color_if_true='#90EF90', color_if_false='#FA6B84', 
                              subset=['tendencia', "crossover"]) \
                            .applymap(lambda _: "color: black; font-weight: bold")

        return styler
    
    estilado = set_styles(tabla)

    #dict_to_df_activos_en_filas.to_excel(f"MACD{activos}-tf{timeframe}.xlsx")  
    return tabla_final_activos_en_filas, tabla_final_activos_en_columnas, df_distancia, df_crossover, estilado




def getGoldenDeathSignals(activos, timeframe):
    
    """
    Esta función devuelve los cruces vigentes de las medias móviles exponenciales de 50 y 200 sesiones del listado de activos
    solicitado, así como el sentido del cruce, la fecha de la señal, el precio de la señal, el rendimiento acumulado a la fecha
    y el rendimiento acumulado, todo ordenado en forma descendente por cantidad de días transcurridos desde la señal.
    
    # Inputs:
        - activos: es una lista de activos a analizar. Ej.: adrs = ["GGAL", "BBAR", "LOMA", "CEPU"]. También hay sets de activos preconfigurados en la función:
            - "lideres": si se desea cargar todo el panel lider de Argentina;
            - "general": si se desea cargar todo el panel general de Argentina;
            - "adrs": si se desea cargar todo el listado de ADRs argentinos (en usd);
            - "sectors": si se desea cargar el listado de sectores de la economía (ETFs representativos de USA);
            - "cedears": si se desea cargar todo el listado de certificados de depósito de activos del exterior que cotizan en Argentina (puede demorar bastante).
        - timeframe: es la duración de cada sesión/vela (compresión temporal). Puede ser "diario", "semanal" o "mensual".
    
    # Outputs:
        - goldendeath_cross_signals: diccionario con las señales vigentes;
        - goldendeath_cross_signals_df: tabla con las señales vigentes;
        - goldendeath_cross_signals_df_formateado: tabla con las señales vigentes formateada con escala de colores según rendimiento
        acumulado anualizado desde la señal.
        
    """
    import pandas as pd
    import pandas_ta as ta
    import yfinance as yf
    
    if timeframe == "diario":
        interval = "1d"
        period = "3y"
    elif timeframe == "semanal":
        interval = "1wk"
        period = "10y"
    elif timeframe == "mensual":
        interval = "1mo"
        period = "15y"
    else:
        return "timeframe incorrecto, elegir 'diario', 'semanal' o 'mensual'"
    
    data = getDataYfMulti(activos, tipo = "period", interval = interval, period = period, swap = False)["Close"]
    data = pd.DataFrame(data)
    data.dropna(inplace = True)

    # Defino datos de estrategia
    MyStrategy = ta.Strategy(
    name = "goldenCross",
    ta = [
        {"kind" : "ema", "close" : "close", "length" : 50},
         {"kind" : "ema", "close" : "close", "length" : 200},
    ]
    )

    df_distancia = pd.DataFrame(index = data.index)  # Es el feature numerico (EMA50/EMA200 - 1) (Variación no porcentual)
    df_crossover = pd.DataFrame(index = data.index)  # Es el feature categorico de crossover de tend: si últimos 3 valores de df_distancia son crecientes, validamos tendencia creciente.            

    goldendeath_cross_signals = {}

    for activo in data.columns:
        try:
            # Iteramos por cada activo
            # Armamos el dataframe OHLC y calculamos componentes del MACD
            datat = pd.DataFrame(data[activo])
            datat.rename(columns = {activo : "Close"}, inplace = True)   # Tengo que renombrar el ticker por "Close", sino no se calcula la MACD.
            datat.ta.strategy(MyStrategy)
            datat.dropna(inplace = True)
            datat["distancia"] = (datat["EMA_50"] / datat["EMA_200"]) - 1

            # Obtenemos el precio actual
            actual_price = round(datat["Close"][-1],2)
            # Determinamos posición y cambios de las EMA de 50 y 200:
            datat["posicionEMAs"] = np.where(datat["distancia"] > 0, 1, -1)
            datat["posicionEMAs_change"] = np.where(datat["posicionEMAs"] > datat["posicionEMAs"].shift(1), "Compra", 
                                                        np.where(datat["posicionEMAs"] < datat["posicionEMAs"].shift(1), "Venta", ""))


            # 1) Caso de Long, calculamos estadísticas:
            if datat["posicionEMAs"][-1] > 0:
                last_buy_signal_crossover = datat[datat["posicionEMAs_change"] == "Compra"].iloc[-1]
                price_lbs_crossover = round(last_buy_signal_crossover["Close"],2)
                date_lbs_crossover = datetime(last_buy_signal_crossover.name.year, 
                                                    last_buy_signal_crossover.name.month, 
                                                    last_buy_signal_crossover.name.day)
                days_since_bs_crossover = (datetime.today() - date_lbs_crossover).days
                rend_since_bs_crossover = round(((actual_price / price_lbs_crossover) - 1) * 100, 2)
                rend_since_bs_annualized_crossover = round((((1 + ((actual_price / price_lbs_crossover) - 1)) ** (365/days_since_bs_crossover)) - 1) * 100,2)
                goldendeath_cross_signals[activo] = {"type" : "golden", 
                                                        "precioSeñal" : price_lbs_crossover, 
                                                        "fechaSeñal": date_lbs_crossover.strftime("%Y-%m-%d"), 
                                                        "diasDesdeSeñal" : days_since_bs_crossover,
                                                        "precioActual" : round(actual_price,2),
                                                        "rendAcumDesdeSeñal":  rend_since_bs_crossover, 
                                                        "rendAnualDesdeSeñal" : rend_since_bs_annualized_crossover}


            # 2) Caso de Short, calculamos estadísticas:     
            elif datat["posicionEMAs"][-1] < 0:
                last_sell_signal_crossover = datat[datat["posicionEMAs_change"] == "Venta"].iloc[-1]
                price_lss_crossover = round(last_sell_signal_crossover["Close"],2)
                date_lss_crossover = datetime(last_sell_signal_crossover.name.year, 
                                                last_sell_signal_crossover.name.month, 
                                                last_sell_signal_crossover.name.day)
                days_since_ss_crossover = (datetime.today() - date_lss_crossover).days
                rend_since_ss_crossover = round(((price_lss_crossover / actual_price) - 1) * 100, 2)
                rend_since_ss_annualized_crossover = round((((1 + ((price_lss_crossover / actual_price) - 1)) ** (365/days_since_ss_crossover)) - 1) * 100,2)
                goldendeath_cross_signals[activo] = {"type" : "death", 
                                                "precioSeñal" : price_lss_crossover, 
                                                "fechaSeñal": date_lss_crossover.strftime("%Y-%m-%d"), 
                                                "diasDesdeSeñal" : days_since_ss_crossover,
                                                "precioActual" : round(actual_price,2),
                                                "rendAcumDesdeSeñal":  rend_since_ss_crossover, 
                                                "rendAnualDesdeSeñal" : rend_since_ss_annualized_crossover}
            print(f"{activo} - procesado!")
        except:
            print(f"{activo} - no pudo ser procesado")
            pass

    goldendeath_cross_signals_df = pd.DataFrame(goldendeath_cross_signals).transpose().sort_values(by = "diasDesdeSeñal", ascending = True)
    goldendeath_cross_signals_df_formateado = goldendeath_cross_signals_df.style.background_gradient(axis=0, gmap=goldendeath_cross_signals_df['rendAnualDesdeSeñal'], 
                                                                                         cmap='RdYlGn') \
    .format('{:.2f}%', subset=["rendAcumDesdeSeñal", "rendAnualDesdeSeñal"]) \
    .format('{:.2f}', subset=["precioSeñal"])
    
    
    return goldendeath_cross_signals, goldendeath_cross_signals_df, goldendeath_cross_signals_df_formateado


In [4]:
tabla, _, df_dist, df_cross, tabla_est = analyzeGoldenDeath(activos = "adrs", timeframe = "diario")

[TF diario] Procesando 1 de 15 activos ...
[TF diario] Procesando 2 de 15 activos ...
[TF diario] Procesando 3 de 15 activos ...
[TF diario] Procesando 4 de 15 activos ...
[TF diario] Procesando 5 de 15 activos ...
[TF diario] Procesando 6 de 15 activos ...
[TF diario] Procesando 7 de 15 activos ...
[TF diario] Procesando 8 de 15 activos ...
[TF diario] Procesando 9 de 15 activos ...
[TF diario] Procesando 10 de 15 activos ...
[TF diario] Procesando 11 de 15 activos ...
[TF diario] Procesando 12 de 15 activos ...
[TF diario] Procesando 13 de 15 activos ...
[TF diario] Procesando 14 de 15 activos ...
[TF diario] Procesando 15 de 15 activos ...


In [10]:
# analyzeGoldenDeath - output - tabla con estado de situación (tendencia = posición de la MM50 respecto a la MM200 y crossover = posición de la distancia entre
# dichas MM y la media móvil de las distancias (suavización, simil línea MACD)
tabla_est

,tendencia,crossover
BBAR,Alcista,Alcista
BMA,Alcista,Alcista
CEPU,Alcista,Alcista
CRESY,Alcista,Alcista
EDN,Alcista,Alcista
GGAL,Alcista,Alcista
IRS,Alcista,Alcista
LOMA,Alcista,Alcista
PAM,Alcista,Alcista
SUPV,Alcista,Alcista


In [11]:
# analyzeGoldenDeath - output - tabla anterior sin estilos
tabla

,tendencia,crossover
BBAR,Alcista,Alcista
BMA,Alcista,Alcista
CEPU,Alcista,Alcista
CRESY,Alcista,Alcista
EDN,Alcista,Alcista
GGAL,Alcista,Alcista
IRS,Alcista,Alcista
LOMA,Alcista,Alcista
PAM,Alcista,Alcista
SUPV,Alcista,Alcista


In [12]:
# analyzeGoldenDeath - output - tabla con las distancias de la MM50d a la MM200d
df_dist

,BBAR,BMA,CEPU,CRESY,EDN,GGAL,IRS,LOMA,PAM,SUPV,TEO,TGS,TS,TX,YPF
Date,,,,,,,,,,,,,,,
2021-11-05,0.174010,0.099919,0.314934,-0.046963,0.383080,0.192822,0.090608,0.061349,0.153856,0.122290,-0.044330,0.077542,0.039805,0.119037,0.021742
2021-11-08,0.172326,0.098847,0.316249,-0.047788,0.378289,0.194790,0.091054,0.060810,0.153470,0.121014,-0.043121,0.077612,0.044110,0.114625,0.019853
2021-11-09,0.171567,0.098231,0.317401,-0.048243,0.371265,0.197132,0.091102,0.060703,0.152646,0.120439,-0.041623,0.077994,0.048290,0.110434,0.019470
2021-11-10,0.171184,0.097900,0.317500,-0.049131,0.364275,0.199422,0.090705,0.060790,0.151935,0.119289,-0.039301,0.078438,0.049979,0.106684,0.018064
2021-11-11,0.170323,0.098114,0.316318,-0.050273,0.355394,0.200687,0.090357,0.060976,0.151610,0.117734,-0.037022,0.080043,0.051424,0.103671,0.016072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-16,0.070346,0.167828,0.239375,0.165341,0.259033,0.125944,0.099831,0.049315,0.128737,0.204609,0.155714,0.123910,0.054931,0.045762,0.167945
2024-01-17,0.069134,0.168789,0.241237,0.163959,0.257246,0.125347,0.098471,0.048794,0.128146,0.206213,0.153494,0.123778,0.052335,0.044463,0.164886
2024-01-18,0.068381,0.170237,0.242778,0.162463,0.257879,0.125539,0.097446,0.048875,0.127584,0.209171,0.151736,0.123597,0.050088,0.043097,0.162327


In [13]:
# analyzeGoldenDeath - output - tabla con las posiciones de la distancia de la MM50d respecto a la MM200d respecto a la MM de las distancias
df_cross

,BBAR,BMA,CEPU,CRESY,EDN,GGAL,IRS,LOMA,PAM,SUPV,TEO,TGS,TS,TX,YPF
Date,,,,,,,,,,,,,,,
2021-11-05,Bajista,Bajista,Bajista,Bajista,Bajista,Bajista,Bajista,Bajista,Bajista,Bajista,Bajista,Bajista,Bajista,Bajista,Bajista
2021-11-08,Bajista,Bajista,Alcista,Alcista,Bajista,Alcista,Alcista,Bajista,Bajista,Bajista,Bajista,Alcista,Alcista,Bajista,Bajista
2021-11-09,Bajista,Bajista,Alcista,Alcista,Bajista,Alcista,Alcista,Bajista,Bajista,Bajista,Bajista,Alcista,Alcista,Bajista,Bajista
2021-11-10,Bajista,Bajista,Alcista,Alcista,Bajista,Alcista,Bajista,Bajista,Bajista,Bajista,Bajista,Alcista,Alcista,Bajista,Bajista
2021-11-11,Bajista,Bajista,Bajista,Alcista,Bajista,Alcista,Bajista,Alcista,Bajista,Bajista,Bajista,Alcista,Alcista,Bajista,Bajista
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-16,Alcista,Alcista,Alcista,Alcista,Alcista,Alcista,Alcista,Alcista,Alcista,Alcista,Alcista,Alcista,Bajista,Alcista,Alcista
2024-01-17,Alcista,Alcista,Alcista,Alcista,Alcista,Alcista,Alcista,Alcista,Alcista,Alcista,Alcista,Alcista,Bajista,Alcista,Alcista
2024-01-18,Alcista,Alcista,Alcista,Alcista,Alcista,Alcista,Alcista,Alcista,Alcista,Alcista,Bajista,Alcista,Bajista,Alcista,Alcista


In [14]:
dic, tabla, tabla_est = getGoldenDeathSignals(activos = "adrs", timeframe = "diario")

BBAR - procesado!
BMA - procesado!
CEPU - procesado!
CRESY - procesado!
EDN - procesado!
GGAL - procesado!
IRS - procesado!
LOMA - procesado!
PAM - no pudo ser procesado
SUPV - procesado!
TEO - procesado!
TGS - procesado!
TS - procesado!
TX - procesado!
YPF - procesado!


In [16]:
# getGoldenDeathSignals - output - diccionario/JSON
dic

{'BBAR': {'type': 'golden',
  'precioSeñal': 5.24,
  'fechaSeñal': '2023-11-24',
  'diasDesdeSeñal': 60,
  'precioActual': 5.58,
  'rendAcumDesdeSeñal': 6.49,
  'rendAnualDesdeSeñal': 46.59},
 'BMA': {'type': 'golden',
  'precioSeñal': 14.62,
  'fechaSeñal': '2022-09-12',
  'diasDesdeSeñal': 498,
  'precioActual': 30.22,
  'rendAcumDesdeSeñal': 106.7,
  'rendAnualDesdeSeñal': 70.27},
 'CEPU': {'type': 'golden',
  'precioSeñal': 3.31,
  'fechaSeñal': '2022-07-28',
  'diasDesdeSeñal': 544,
  'precioActual': 9.29,
  'rendAcumDesdeSeñal': 180.66,
  'rendAnualDesdeSeñal': 99.85},
 'CRESY': {'type': 'golden',
  'precioSeñal': 5.99,
  'fechaSeñal': '2022-12-29',
  'diasDesdeSeñal': 390,
  'precioActual': 9.28,
  'rendAcumDesdeSeñal': 54.92,
  'rendAnualDesdeSeñal': 50.64},
 'EDN': {'type': 'golden',
  'precioSeñal': 6.86,
  'fechaSeñal': '2022-08-12',
  'diasDesdeSeñal': 529,
  'precioActual': 20.29,
  'rendAcumDesdeSeñal': 195.77,
  'rendAnualDesdeSeñal': 111.33},
 'GGAL': {'type': 'golden',

In [17]:
# getGoldenDeathSignals - output - tabla resumen
tabla

,type,precioSeñal,fechaSeñal,diasDesdeSeñal,precioActual,rendAcumDesdeSeñal,rendAnualDesdeSeñal
SUPV,golden,3.34,2023-12-04,50,4.18,25.15,414.33
TX,golden,40.95,2023-11-28,56,38.65,-5.62,-31.39
IRS,golden,9.01,2023-11-27,57,8.69,-3.55,-20.67
BBAR,golden,5.24,2023-11-24,60,5.58,6.49,46.59
TGS,golden,13.09,2023-11-21,63,14.78,12.91,102.08
YPF,golden,14.83,2023-11-21,63,15.64,5.46,36.08
GGAL,golden,14.65,2023-11-20,64,18.18,24.1,242.53
TS,golden,31.89,2023-07-19,188,31.98,0.28,0.55
TEO,golden,5.16,2023-01-05,383,7.83,51.74,48.8
CRESY,golden,5.99,2022-12-29,390,9.28,54.92,50.64


In [18]:
# getGoldenDeathSignals - output - tabla resumen con estilos
tabla_est

,type,precioSeñal,fechaSeñal,diasDesdeSeñal,precioActual,rendAcumDesdeSeñal,rendAnualDesdeSeñal
SUPV,golden,3.34,2023-12-04,50,4.180000,25.15%,414.33%
TX,golden,40.95,2023-11-28,56,38.650000,-5.62%,-31.39%
IRS,golden,9.01,2023-11-27,57,8.690000,-3.55%,-20.67%
BBAR,golden,5.24,2023-11-24,60,5.580000,6.49%,46.59%
TGS,golden,13.09,2023-11-21,63,14.780000,12.91%,102.08%
YPF,golden,14.83,2023-11-21,63,15.640000,5.46%,36.08%
GGAL,golden,14.65,2023-11-20,64,18.180000,24.10%,242.53%
TS,golden,31.89,2023-07-19,188,31.980000,0.28%,0.55%
TEO,golden,5.16,2023-01-05,383,7.830000,51.74%,48.80%
CRESY,golden,5.99,2022-12-29,390,9.280000,54.92%,50.64%
